## Script for creating Sankey diagrams for C-THRU petrochemical mappings paper

In [76]:
import re
import numpy as np
import pandas as pd
from floweaver import *
import ipywidgets as ipw
from ipysankeywidget import SankeyWidget

### Load mapping data

In [435]:
## Input params
mapping_file = '../data/input_data/country_summaries_w_uncertainties.parquet'
year = 2030
gas = 'CO2e_100a'

data_cols = ['Region', 'Product type', 'Product group', 'Type', 'Gas', 'PRODUCT', 'COUNTRY/TERRITORY', str(year), str(year)+'_sigma']

In [436]:
## Load and filter dataframes
def filter_df(df, cols, filters):
    for col, filt in zip(cols, filters):
        df = df[[i in filt for i in df[col]]]
    return df

# Load
data = pd.read_parquet(mapping_file)
dp_negs_filt = pd.read_csv('negative_direct_processes.csv')
dp_pos_filt = pd.read_csv('positive_direct_processes.csv')

# Filter
data_filt = filter_df(data[data_cols], ['Gas', 'Type'], [[gas], ['Imports', 'Indirect Energy Use', 'Direct Energy Use', 'Feedstock', 'Primary chemicals', 'Organic chemicals', 'Other intermediates']])
data_filt = data_filt[data_filt['Product group']!=data_filt['Type']]
data_filt = data_filt.groupby(data_cols[:-4]).sum().reset_index()

# Rename for Sankey stuff
data_filt.rename(columns={
    'Product group':'target',
    'Type':'source',
    'Product type':'Level',
    str(year):'value',
    str(year)+'_sigma':'error'
}, inplace=True)
data_filt.drop(columns=['Gas'], inplace=True)
data_filt['type'] = data_filt['source']

# Change type of imports to type of product group it is imported to
data_filt['type'] = [i if i!='Imports' else data_filt['target'].iloc[num] for num, i in enumerate(data_filt['type'])]

data_filt.head()

,Region,Level,target,source,value,error,type
0,Africa,Downstream,N-fertilisers,Direct Energy Use,0.000000,0.000000,Direct Energy Use
1,Africa,Downstream,N-fertilisers,Feedstock,20735.333006,622.059990,Feedstock
2,Africa,Downstream,N-fertilisers,Indirect Energy Use,3942.014115,304.643065,Indirect Energy Use
3,Africa,Downstream,N-fertilisers,Organic chemicals,0.000000,0.000000,Organic chemicals
4,Africa,Downstream,N-fertilisers,Other intermediates,0.000000,0.000000,Other intermediates


Adjust captured flows -- I'm not sure if this is the best way

In [437]:
dp_negs_filt["target"] = "Captured"
dp_negs_filt["source"] = "N-fertilisers"

In [438]:
dp_negs_filt

,Region,Level,source,Product,value,error,type,target
0,Africa,Downstream,N-fertilisers,UREA,8534.613237,0.0,Direct Process,Captured
1,Europe,Downstream,N-fertilisers,UREA,6724.898002,0.0,Direct Process,Captured
2,Former USSR,Downstream,N-fertilisers,UREA,12151.479021,0.0,Direct Process,Captured
3,Latin America,Downstream,N-fertilisers,UREA,2231.422777,0.0,Direct Process,Captured
4,Middle East,Downstream,N-fertilisers,UREA,17225.232617,0.0,Direct Process,Captured
5,North America,Downstream,N-fertilisers,UREA,11410.138412,0.0,Direct Process,Captured
6,North East Asia,Downstream,N-fertilisers,UREA,50197.633067,0.0,Direct Process,Captured
7,Oceania,Downstream,N-fertilisers,UREA,304.904446,0.0,Direct Process,Captured
8,South Asia,Downstream,N-fertilisers,UREA,24211.508691,0.0,Direct Process,Captured
9,World,Downstream,N-fertilisers,UREA,132991.830270,0.0,Direct Process,Captured


## Define helper functions

In [439]:
## Create functions for writing values of flows in Sankey labels
def break_string(x, words = 4, fert_skips=3):
    spaces = [i.start() for i in re.finditer(' ', x)]
    
    if len(spaces) >= words:
        return x[0].upper() + x[1:spaces[words - 1]] + '\n' + x[spaces[words - 1]+1:]
    if 'N-' in x:
        if fert_skips == 2:
            return ' \n .\n .  '+x[0].upper() + x[1:]
        else: return ' \n .\n .\n .  '+x[0].upper() + x[1:]
    else:
        return x[0].upper() + x[1:]
    
def get_Evalues_to_target(flows, process, shorten=False):
    value = round(sum(flows.loc[flows.target == process, 'value'])/1E3 , 1)
    err = round(sum(flows.loc[flows.target == process, 'error'])/1E3 , 1)
    if err==0:
        err=err+0.1
    if value < 10:
        return ' ' + str(value)+"±"+str(err)+' Mt'
    if round(err, 0) == 0:
        return ' ' + str(int(round(value,0)))+"±"+str(err)+' Mt'
    return ' ' + str(int(round(value,0)))+"±"+str(int(round(err,0))) + ' Mt'

def get_Evalues_to_source(flows,process):
    value = round(sum(flows.loc[flows['source'] == process, 'value'])/1E3 , 2)
    err = round(sum(flows.loc[flows['source'] == process, 'error'])/1E3 , 2)
    if value < 10:
        return '\n' + str(value)+"±"+str(err)+' Mt'
    if round(err, 0) == 0:
        return '\n' + str(int(round(value,0)))+"±"+str(err)+' Mt'    
    return '\n' + str(int(value))+"±"+str(int(err)) + ' Mt'


## Define Sankey properties

In [440]:
## Define order of sources and targets
source_order = ['Direct Energy Use', 'Direct Process', 'Feedstock', 'Indirect Energy Use']
downstream_order = ['N-fertilisers', 'Thermoplastics', 'Solvents, additives & explosives', 'Thermosets, fibre & elastomers', 'Other downstream']

In [441]:
def sankey_features(sankey_table, source_order, downstream_order_used, details=False, fert_skips=3):

    # Get values for labels
    source_vals = sankey_table.groupby('source').sum(numeric_only=True).reset_index()
    target_vals = sankey_table.groupby('target').sum(numeric_only=True).reset_index()
    imports_value = sankey_table[sankey_table['source']=='Imports'].sum(numeric_only=True)
    imports_error = np.ceil(0.1*imports_value['error']/1E3)
    dir_proc_value = sankey_table[sankey_table['target']=='Captured'].sum(numeric_only=True)

    # Create partitions with data labels
    if details:
        shortened_source = source_order.copy()
        shortened_downstream = downstream_order_used.copy()
    else: 
        shortened_source = ['DEU', 'DP', 'FS', 'IEU']
        shortened_downstream = ['N-Fs', 'TPs', 'SA&E', 'TF&E', 'Other & exports\n']

    source_partition = Partition(tuple([Group(break_string(s, words=5) + get_Evalues_to_source(source_vals, i),
                                              (('source', (i,)),)) for i, s in zip(source_order, shortened_source)]))

    downstream_partition = Partition(tuple([Group(break_string(s, words=5, fert_skips=fert_skips) + get_Evalues_to_target(target_vals, i),
                                                  (('process', (i,)),)) for i, s in zip(downstream_order_used, shortened_downstream)]))
    # downstream_partition = Partition(tuple([Group(break_string(s, words=5) + get_Evalues_to_target(target_vals, i),
    #                                               (('target', (i,)),)) for i, s in zip(downstream_order_used, shortened_downstream)]))

    print(downstream_partition)# = Partition.Simple('process', shortened_downstream)
    #

    # Define nodes
    nodes = {
        'sources': ProcessGroup('type == "source"', source_partition, title='Sources' if details else ""),
        'imports': ProcessGroup(['Imports'], title='Imports\n'+str(int(imports_value['value']/1E3))+"±"+str(int(imports_error))+' Mt'),
        'pc': ProcessGroup(['Primary chemicals'], title='Primary\nchemicals' if details else 'PC'),
        'oc': ProcessGroup(['Organic chemicals'], title='Organic\nchemicals' if details else 'OC'),
        'other_ints': ProcessGroup(['Other intermediates'], title='Other\nintermediates' if details else 'OI'),
        'downstream': ProcessGroup('type == "downstream"', downstream_partition, title='Downstream' if details else ""),
        'captured': ProcessGroup(['Captured'], title='Captured\n'+str(int(dir_proc_value['value']/1E3))+"±"+str(int(np.ceil(0.1*dir_proc_value['value']/1E3)))+' Mt'),
        'up1': Waypoint(title=''),
        'up2': Waypoint(title=''),
        'up3': Waypoint(title=''),
        'pcw1': Waypoint(title=''),
        'pcw2': Waypoint(title=''),
        'ocw1': Waypoint(title=''),
        'imp1': Waypoint(title=''),
        'imp2': Waypoint(title=''),
    }



    # Define ordering
    ordering = [
        [[], ['sources', 'imports']],
        [[], ['up1', 'pc', 'imp1']],
        [[], ['up2','pcw1','oc', 'imp2']],
        [[], ['up3','pcw2','ocw1','other_ints']],
        [[], ['downstream']],
        [['captured'], []]
    ]

    # Define bundles
    bundles = [
              Bundle('sources','pc'),
              Bundle('sources','oc', waypoints=['up1']),
              Bundle('sources','other_ints', waypoints=['up1', 'up2']),
              Bundle('sources','downstream', waypoints=['up1', 'up2', 'up3']),
              Bundle('imports','pc'),
              Bundle('imports','oc', waypoints=['imp1']),
              Bundle('imports','other_ints', waypoints=['imp1', 'imp2']),
              Bundle('pc','oc'),
              Bundle('pc','other_ints', waypoints=['pcw1']),
              Bundle('pc','downstream',  waypoints=['pcw1', 'pcw2']),
              Bundle('oc','pc'),
              Bundle('oc','other_ints'),
              Bundle('oc','downstream',  waypoints=['ocw1']),
              Bundle('other_ints','downstream'),
              Bundle('downstream','captured'),
    ]
    return {'nodes':nodes,'ordering':ordering,'bundles':bundles}

palette = {
    'Feedstock': '#E66070',
    'Direct Process':'#498DE6',
    'Direct Energy Use': '#6EE588',
    'Indirect Energy Use': '#E6A730',
    'Organic chemicals': '#888888',
    'Other intermediates': '#A055E6',
    'Primary chemicals': '#444444'
}

In [442]:
def create_diagram(region, data, neg_dps, pos_dps, source_order, downstream_order, width=400, left_marg=75, right_marg=120, details=False, fert_skips=3):

    region_data = filter_df(data, ['Region'], [region])
    dp_negs_reg = filter_df(neg_dps, ['Region'], [region])
    dp_pos_reg = filter_df(pos_dps, ['Region'], [region])
    
    sankey_table = pd.concat((region_data, dp_negs_reg, dp_pos_reg))
    sankey_table = sankey_table[sankey_table["value"] > 1e-9]

    # Create processes
    processes = pd.DataFrame(columns=['id', 'type'])
    processes['id'] = np.concatenate((source_order, downstream_order))
    processes['type'] = np.concatenate((['source']*len(source_order), ['downstream']*len(downstream_order)))

    # Draw Sankey
    dataset = Dataset(sankey_table, dim_process=processes.set_index('id'))
    features = sankey_features(sankey_table, source_order, downstream_order, details, fert_skips=fert_skips)
    flow_partition = Partition.Simple("type", sankey_table['type'].unique())
    sdd = SankeyDefinition(features['nodes'], features['bundles'], features['ordering'], flow_partition = flow_partition)
    result = weave(sdd, dataset, palette=palette) \
        .to_widget(width=width, height=250, margins={'right':right_marg, 'left':left_marg, 'top': 11}, align_link_types=False, debugging=True)
    return sankey_table, result, features

In [443]:
## Loop for regional diagrams
region_groups = [['Europe'], ['Middle East'], ['North America'], ['North East Asia'], ['South Asia'], ['Africa', 'Former USSR', 'Latin America', 'Oceania']]

## Define sources and targets
source_order = ['Direct Energy Use', 'Direct Process', 'Feedstock', 'Indirect Energy Use']
downstream_order = ['N-fertilisers', 'Thermoplastics', 'Solvents, additives & explosives', 'Thermosets, fibre & elastomers', 'Other downstream & exports']

reg_values, reg_errors = [], []
reg_diagrams = {}
for num, (region, skip) in enumerate(zip(region_groups, [2,2,2,2,2,3])):
    sankey_table, diag, features = create_diagram(region, data_filt, dp_negs_filt, dp_pos_filt, source_order, downstream_order, details=False, fert_skips=skip)
    reg_diagrams.update({region[0]:diag})

    captured_val = sankey_table[sankey_table['target']=='Captured'].sum()['value']
    summary_vals = filter_df(sankey_table,  ['target'], [downstream_order]).sum(numeric_only=True)
    reg_values = reg_values + ['{:,}'.format(int(((summary_vals['value']-captured_val)/1E3)))]
    reg_errors = reg_errors + ['{:,}'.format(int(summary_vals['error']/1E3))]
    
#diag.children[0].scale = 0.0003
diag

Partition(groups=(Group(label=' \n .\n .  N-Fs 60±10 Mt', query=(('process', ('N-fertilisers',)),)), Group(label='TPs 110±24 Mt', query=(('process', ('Thermoplastics',)),)), Group(label='SA&E 2.4±0.8 Mt', query=(('process', ('Solvents, additives & explosives',)),)), Group(label='TF&E 38±6 Mt', query=(('process', ('Thermosets, fibre & elastomers',)),)), Group(label='Other & exports\n 87±18 Mt', query=(('process', ('Other downstream & exports',)),))))
Partition(groups=(Group(label=' \n .\n .  N-Fs 76±11 Mt', query=(('process', ('N-fertilisers',)),)), Group(label='TPs 92±27 Mt', query=(('process', ('Thermoplastics',)),)), Group(label='SA&E 0.4±0.2 Mt', query=(('process', ('Solvents, additives & explosives',)),)), Group(label='TF&E 13±2 Mt', query=(('process', ('Thermosets, fibre & elastomers',)),)), Group(label='Other & exports\n 32±6 Mt', query=(('process', ('Other downstream & exports',)),))))
Partition(groups=(Group(label=' \n .\n .  N-Fs 75±13 Mt', query=(('process', ('N-fertilisers',

In [444]:
## Create world diagram
## Define sources and targets
source_order = ['Direct Energy Use', 'Direct Process', 'Feedstock', 'Indirect Energy Use']
downstream_order = ['N-fertilisers', 'Thermoplastics', 'Solvents, additives & explosives', 'Thermosets, fibre & elastomers', 'Other downstream']

world_table, world_diag, features = create_diagram('World', data_filt, dp_negs_filt, dp_pos_filt,source_order, downstream_order,
                                          width=800, left_marg=150, right_marg=250, details=True, fert_skips=2)
captured_val = world_table[world_table['target']=='Captured'].sum()['value']
summary_vals = filter_df(world_table, ['target'], [downstream_order]).sum(numeric_only=True)
world_val, world_err = '{:,}'.format(int(((summary_vals['value']-captured_val)/1E3))), '{:,}'.format(int(summary_vals['error']/1E3))

Partition(groups=(Group(label=' \n .\n .  N-fertilisers 713±116 Mt', query=(('process', ('N-fertilisers',)),)), Group(label='Thermoplastics 1218±286 Mt', query=(('process', ('Thermoplastics',)),)), Group(label='Solvents, additives & explosives 19±6 Mt', query=(('process', ('Solvents, additives & explosives',)),)), Group(label='Thermosets, fibre & elastomers 573±112 Mt', query=(('process', ('Thermosets, fibre & elastomers',)),)), Group(label='Other downstream 343±69 Mt', query=(('process', ('Other downstream',)),))))


In [445]:
world_val

'2,731'

In [446]:
## Create HTML visualisation box

region_filt_names = region_groups[:-1]+[['Other']]
letters = ['b','c','d','e','f','g']

for num, (reg, let) in enumerate(zip(region_groups[::2], letters[::2])):
    val1, val2 = reg_values[num*2], reg_values[num*2+1]
    err1, err2 = reg_errors[num*2], reg_errors[num*2+1]
    if num==0:
        text = "<pre class=tab><big><b>     "+let+"</b>      "+reg[0]+": "+str(val1)+"&plusmn;"+err1+" Mt                <b>"+letters[num*2+1]+"</b>       "+region_filt_names[num*2+1][0]+": "+str(val2)+"&plusmn;"+err2+" Mt</big></pre>"
    elif num==1:
        text = "<pre class=tab><big><b>     "+let+"</b>  "+reg[0]+": "+str(val1)+"&plusmn;"+err1+" Mt            <b>"+letters[num*2+1]+"</b>   "+region_filt_names[num*2+1][0]+": "+str(val2)+"&plusmn;"+err2+" Mt</big></pre>"
    else:
        text = "<pre class=tab><big><b>     "+let+"</b>    "+reg[0]+": "+str(val1)+"&plusmn;"+err1+" Mt               <b>"+letters[num*2+1]+"</b>       "+region_filt_names[num*2+1][0]+": "+str(val2)+"&plusmn;"+err2+" Mt</big></pre>"

    globals()['name'+str(num)] = ipw.HTML(value=text)
    globals()['row'+str(num)] = ipw.HBox((reg_diagrams[reg[0]], reg_diagrams[region_groups[num*2+1][0]]))

name_world = ipw.HTML("<pre class=tab><big><b>    a</b>                        World: "+str(world_val)+"&plusmn;"+str(world_err)+" Mt")
final = ipw.VBox((name_world, world_diag, name0, row0, name1, row1, name2, row2))

In [448]:
final